In [1]:
import numpy as np
import gradio as gr
import tensorflow as tf
from tensorflow import keras
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
window_name = "window"
labels = ['Sagittal','Frontal']

In [3]:
Sag_Fron_Class_Model = keras.models.load_model('Image_Class_sagittal_frontal_model_7.tflearn')

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [5]:
def video_flip(video):
    Knee_Angles = []
    Knee_Projection_Angles = []
    Total_Time = []
    Sagittal_Num = []
    Frontal_Num = []
    Sag_Fron_Class = []
    Sagittal_Percent = []
    Frontal_Percent = []
    
    cap = cv2.VideoCapture(video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fps = round(fps, 0)
       
    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()

    cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while (True):
            ret, frame = cap.read()
            if not ret:
                print("Reached end of video, exiting.")
                break
        
            #Sagittal/Frontal Plane Classification
            size = (456,608)
            image2 = cv2.resize(frame,size)
            image2 = image2.reshape(1,456,608,3)
            Sag_Fron_Class_pred = Sag_Fron_Class_Model.predict(image2)
            Sag_Fron_Class_pred = np.argmax(np.round(Sag_Fron_Class_pred),axis=1)
            Sag_Fron_Class.extend(Sag_Fron_Class_pred)
        
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
      
            # Make detection
            results = pose.process(image)
    
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
            
                # Get coordinates
                RShoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                RHip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                RKnee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                RAnkle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                RToe = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
                Vert_Knee_Line = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y+.1]
                Knee_Angle = calculate_angle(Vert_Knee_Line, RKnee, RAnkle)
                
                Knee_Angles.append(Knee_Angle)
                #find points to find frontal plane knee projections angle
                Kne_Ank_Diff = (RKnee[0]-RAnkle[0],RKnee[1]-RAnkle[1])
                Knee_Projection = (RKnee[0]+Kne_Ank_Diff[0],RKnee[1]+Kne_Ank_Diff[1])
                Knee_Projection_Angle = calculate_angle(RKnee,RHip,Knee_Projection)
                Knee_Projection_Angles.append(Knee_Projection_Angle)
                
                knee_line.append(Vert_Knee_Line)
                              
            except:
                pass
        
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                        )
            cv2.imshow(window_name, image)
            
            if ret == True:
                cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
                cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,
                        cv2.WINDOW_FULLSCREEN)
                if cv2.waitKey(30) & 0xFF == ord('q'):
                    print("Exit requested")
                    break

        cap.release()
        cv2.destroyAllWindows()
            
    for i in range(len(Sag_Fron_Class)):
        if Sag_Fron_Class[i] == 0:
            Sagittal_Count = 1
            Sagittal_Num.append(Sagittal_Count)
        if Sag_Fron_Class[i] == 1:
            Frontal_Count = 1
            Frontal_Num.append(Frontal_Count)
    Sagittal_Percent = (len(Sagittal_Num))/(len(Sag_Fron_Class))
    Frontal_Percent = (len(Frontal_Num))/(len(Sag_Fron_Class))
    print(Sagittal_Percent)
    
    #df2 = pd.DataFrame(Sagittal_Percent,Frontal_Percent)
    #df2['Frontal'] = list(Frontal_Percent)
    #df2.columns=['Sagittal','Frontal']
        
    if len(Sagittal_Num) > len(Frontal_Num):
        Cam_Angle = 'Sagittal'
    elif len(Frontal_Num) > len(Sagittal_Num):
        Cam_Angle = 'Frontal'
        print(Cam_Angle)
    df=pd.DataFrame(range(len(Knee_Angles)))
    if Cam_Angle == 'Sagittal':
        df['Sagittal Knee Angles'] = Knee_Angles
        df.columns= ['Time','Sagittal Knee Angles']
    if Cam_Angle == 'Frontal':
        df['Frontal Knee Angles'] = Knee_Projection_Angles
        df.columns= ['Time','Frontal Knee Angles']
    #return df
    return (
        df,
        {"Sagittal": Sagittal_Percent, "Frontal": Frontal_Percent},
    )
        #{"Sagittal": pd.DataFrame(list(len(Sagittal_Num)/len(Sag_Fron_Class))), "Frontal": pd.DataFrame(list(len(Frontal_Num)/len(Sag_Fron_Class)))},        

In [6]:
video = gr.inputs.Video(source="upload")
time_series = gr.outputs.Timeseries(x='Time', y=['Knee Angles'])
label = gr.outputs.Label(label="Camera View")

In [7]:
iface = gr.Interface(
    video_flip,
    [
        video],[time_series,label,],)
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://44463.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x1556361f0d0>,
 'http://127.0.0.1:7860/',
 'https://44463.gradio.app')

Reached end of video, exiting.
0.15
Frontal
Reached end of video, exiting.
0.15
Frontal
Reached end of video, exiting.
0.875
Reached end of video, exiting.
0.0
Frontal
